In [1]:
import pandas as pd
import numpy as np
from glob import glob
import numbers

In [2]:
filenames = glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k]*160c.dta')
list_dfs  = [pd.read_stata(f) for f in filenames]

In [3]:
list_dfs = [list_dfs[i].set_index('xwaveid') for i in range(11)]

In [4]:
list_males = [list_dfs[i][list_dfs[i].filter(regex='hgsex$').columns]for i in range(11)]
list_males = [list_dfs[i][list_males[i].iloc[:,0] == '[1] Male'] for i in range(11)]

In [15]:
waves_employed_index = [list_dfs[i].filter(regex='esdtl$').columns for i in range(11)] #
waves_employed = [list_dfs[i][waves_employed_index[i]] for i in range(11)] #waves_employed[0-10] are now created (waves1-11)

In [16]:
males_employed = [list_males[i][waves_employed[i].iloc[:,0] == '[2] Employed PT'] for i in range(11)]

C:\Users\idias\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [17]:
pref1 = [males_employed[i].filter(regex='jbhrcpr$') for i in range(11)]
pref2 = [males_employed[i].filter(regex='jbhrcpr$') for i in range(11)]
pref2 = pref2[1:]

In [18]:
pref1[0]

,ajbhrcpr
xwaveid,
0100003,[2] About the same
0100016,[2] About the same
0100040,[3] More hours
0100059,[3] More hours
0100060,[2] About the same
...,...
0119837,[3] More hours
0119838,[3] More hours
0119839,[2] About the same


In [19]:
diff = [pref1[i].join(pref2[i]) for i in range(10)]
cleaned = [diff[i].dropna(how = 'any') for i in range(10)]

In [20]:
t_minus_1 = [cleaned[i].iloc[:,0].value_counts() for i in range(10)]
t = [cleaned[i].iloc[:,1].value_counts() for i in range(10)]

In [21]:
clean_fewer = [cleaned[i][cleaned[i].iloc[:,0] == '[1] Fewer hours'] for i in range(10)]
clean_fewer = [clean_fewer[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

clean_current = [cleaned[i][cleaned[i].iloc[:,0] == '[2] About the same']for i in range(10)]
clean_current = [clean_current[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

clean_more = [cleaned[i][cleaned[i].iloc[:,0] == '[3] More hours'] for i in range(10)]
clean_more = [clean_more[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

In [22]:
clean_fewer = pd.DataFrame(clean_fewer)
clean_current = pd.DataFrame(clean_current)
clean_more  = pd.DataFrame(clean_more)

In [23]:
clean_unfin = clean_fewer.merge(clean_current, left_index=True, right_index=True)
clean_merged = clean_unfin.merge(clean_more, left_index=True, right_index=True)

In [24]:
unclean = clean_merged.mean(axis=0)
unclean = unclean*100

In [25]:
#_X = prefer to work fewer hours, _Y = current hours are good, _ = Prefer to work more hours

In [26]:
unclean = pd.DataFrame(unclean)

In [27]:
fewer_hrs = unclean[:7]
curr_hrs = unclean[7:14]
more_hrs = unclean[14:]

In [28]:
fewer_hrs.columns = ['Prefer to work fewer hours (t-1)']
curr_hrs.columns = ['Current hours are good (t-1)']
more_hrs.columns = ['Prefer to work more hours (t-1)']

In [29]:
fewer_hrs.index = more_hrs.index
curr_hrs.index = more_hrs.index

In [30]:
final_table = fewer_hrs.join(curr_hrs)
final_table = final_table.join(more_hrs)

In [31]:
final_table.sort_values('Prefer to work fewer hours (t-1)', ascending=False)

,Prefer to work fewer hours (t-1),Current hours are good (t-1),Prefer to work more hours (t-1)
[2] About the same,55.962267,74.396817,35.763182
[1] Fewer hours,27.647303,5.981077,1.973330
[3] More hours,15.955647,19.402071,62.190496
[-3] Dont know,0.434783,0.220036,0.072993
[-10] Non-responding person,0.000000,0.000000,0.000000
[-1] Not asked,0.000000,0.000000,0.000000
[-4] Refused/Not stated,0.000000,0.000000,0.000000


Small extension: The above table is an average of ALL waves (t-1). 